In [2]:
vienna_raw = pd.read_csv("/Users/ar3s/Desktop/flowie_data/data/TRINKBRUNNENOGD.csv")

vienna_raw.head()

,FID,SHAPE,NAME,TYP_NUM,SE_SDO_ROWID,SE_ANNO_CAD_DATA
0,TRINKBRUNNENOGD.2255748,POINT (16.32592771593872 48.212913790547624),Trinkbrunnen,4,2255748,NaN
1,TRINKBRUNNENOGD.2255749,POINT (16.47379576776317 48.23993838096623),Trinkbrunnen,4,2255749,NaN
2,TRINKBRUNNENOGD.2255750,POINT (16.473719310932733 48.239967824396345),Trinkbrunnen,4,2255750,NaN
3,TRINKBRUNNENOGD.2255751,POINT (16.473720828450936 48.23990717622222),Trinkbrunnen,4,2255751,NaN
4,TRINKBRUNNENOGD.2255752,POINT (16.474217693776392 48.24066488215793),Trinkbrunnen,4,2255752,NaN


In [3]:
vienna_raw.tail()

,FID,SHAPE,NAME,TYP_NUM,SE_SDO_ROWID,SE_ANNO_CAD_DATA
1375,TRINKBRUNNENOGD.2254725,POINT (16.450558286904528 48.213169626776136),Auslaufbrunnen,7,2254725,NaN
1376,TRINKBRUNNENOGD.2254726,POINT (16.44350163735615 48.19876716128841),Auslaufbrunnen,7,2254726,NaN
1377,TRINKBRUNNENOGD.2254727,POINT (16.41265487050078 48.22057124820074),Auslaufbrunnen,7,2254727,NaN
1378,TRINKBRUNNENOGD.2254728,POINT (16.341579829475656 48.1897150654744),Auslaufbrunnen,7,2254728,NaN
1379,TRINKBRUNNENOGD.2254729,POINT (16.256040723512957 48.20363941194412),Auslaufbrunnen,7,2254729,NaN


In [4]:
vienna_raw["NAME"].unique()

array(['Trinkbrunnen', 'Trinkbrunnen mit Tränke', 'Spielbrunnen',
       'Sprühnebeldusche', 'Grundwasserbrunnen', 'Wasserspielmöglichkeit',
       'Bodenfontäne', 'Hundetrinkbrunnen', 'Trinkhydrant',
       'Auslaufbrunnen', 'Zierbrunnen'], dtype=object)

In [5]:
vienna_raw["NAME"].value_counts()

NAME
Trinkbrunnen mit Tränke    397
Auslaufbrunnen             341
Trinkbrunnen               273
Trinkhydrant               151
Spielbrunnen                94
Sprühnebeldusche            88
Wasserspielmöglichkeit      20
Bodenfontäne                10
Hundetrinkbrunnen            3
Grundwasserbrunnen           2
Zierbrunnen                  1
Name: count, dtype: int64

In [6]:
vienna_raw.groupby(["NAME", "TYP_NUM"]).size()

NAME                     TYP_NUM
Auslaufbrunnen           7          341
Bodenfontäne             12          10
Grundwasserbrunnen       1            2
Hundetrinkbrunnen        2            3
Spielbrunnen             3           94
Sprühnebeldusche         11          88
Trinkbrunnen             4          273
Trinkbrunnen mit Tränke  5          397
Trinkhydrant             10         151
Wasserspielmöglichkeit   13          20
Zierbrunnen              6            1
dtype: int64

### Info about dataframe from gov.at
There are <b>11 types of water sources</b> in the database.


Translation of names to English:
- Drinking fountain with trough 397<br>
- Flow fountain                 341<br>
- Drinking fountain             273<br>
- Drinking hydrant              151<br>
- Play fountain                 94<br>
- Spray mist shower             88<br>
- Water play facility           20<br>
- Floor fountain                10<br>
- Dog drinking fountain          3<br>
- Groundwater fountain           2<br>
- Ornamental fountain            1<br>

ChatGPT said that <b>Trinkbrunnen</b> are the standard public drinking fountains.<br>
<b>Trinkbrunnen mit Tränke</b> are drinking fountains with an additional dog bowl. Still perfectly safe for humans - extremely common in Vienna. All others are excluded.<br>
This gives us <b>273 + 397 = 670 fountains</b>.

In [7]:
# We keep only Trinkbrunnen (4) + Trinkbrunnen mit Tränke (5)
vienna_filtered = vienna_raw[vienna_raw["TYP_NUM"].isin([4, 5])].copy()

vienna_filtered["NAME"].value_counts()


NAME
Trinkbrunnen mit Tränke    397
Trinkbrunnen               273
Name: count, dtype: int64

In [8]:
def wkt_point_to_lat_lng(wkt_str: str):
    """
    Convert str 'POINT (lon lat)' -> (lat, lng)
    """
    wkt_str = wkt_str.strip()
    coords_str = wkt_str.replace("POINT (", "").replace(")", "")
    lon_str, lat_str = coords_str.split()
    lon = float(lon_str)
    lat = float(lat_str)
    return lat, lon

# Apply to the SHAPE column,then expand into two new columns
vienna_filtered[["lat", "lng"]] = vienna_filtered["SHAPE"].apply(
    lambda s: pd.Series(wkt_point_to_lat_lng(s))
)

vienna_filtered[["SHAPE", "lat", "lng"]].head()


,SHAPE,lat,lng
0,POINT (16.32592771593872 48.212913790547624),48.212914,16.325928
1,POINT (16.47379576776317 48.23993838096623),48.239938,16.473796
2,POINT (16.473719310932733 48.239967824396345),48.239968,16.473719
3,POINT (16.473720828450936 48.23990717622222),48.239907,16.473721
4,POINT (16.474217693776392 48.24066488215793),48.240665,16.474218


In [9]:
# id: TRINKBRUNNENOGD.2255748 -> 2255748
vienna_filtered["id"] = vienna_filtered["FID"].str.split(".").str[-1].astype(int)

# Our own English label for the app 
vienna_filtered["type_label"] = vienna_filtered["NAME"].replace({
    "Trinkbrunnen": "drinking_fountain",
    "Trinkbrunnen mit Tränke": "drinking_fountain_with_trough"
})

# The final clean data
vienna_clean = vienna_filtered[[
    "id",          # numeric id
    "NAME",        # original human-readable name
    "type_label",  # our own label for the app
    "TYP_NUM",     # original type code, just in case
    "lat", "lng"   # coordinates
]].rename(columns={
    "NAME": "name",
    "TYP_NUM": "type_num"
})

vienna_clean.head()


,id,name,type_label,type_num,lat,lng
0,2255748,Trinkbrunnen,drinking_fountain,4,48.212914,16.325928
1,2255749,Trinkbrunnen,drinking_fountain,4,48.239938,16.473796
2,2255750,Trinkbrunnen,drinking_fountain,4,48.239968,16.473719
3,2255751,Trinkbrunnen,drinking_fountain,4,48.239907,16.473721
4,2255752,Trinkbrunnen,drinking_fountain,4,48.240665,16.474218


In [10]:
vienna_clean.tail()

,id,name,type_label,type_num,lat,lng
1087,2254437,Trinkbrunnen,drinking_fountain,4,48.236799,16.404478
1088,2254438,Trinkbrunnen,drinking_fountain,4,48.241611,16.398685
1089,2254439,Trinkbrunnen,drinking_fountain,4,48.245483,16.395727
1090,2254440,Trinkbrunnen,drinking_fountain,4,48.247016,16.393796
1091,2254441,Trinkbrunnen,drinking_fountain,4,48.267780,16.375457


In [11]:
# Saved as CSV
vienna_clean.to_csv("/Users/ar3s/Desktop/flowie_data/data/vienna_fountains_clean.csv", index=False)

# I also save as JSON just in case
vienna_clean.to_json(
    "/Users/ar3s/Desktop/flowie_data/data/vienna_fountains_clean.json",
    orient="records",
    force_ascii=False
)
